In [2]:
# load language model
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ll3_fitting/epoch_329",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True
)

FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [142]:
tokenizer.pad_token = '<|reserved_special_token_250|>'
tokenizer.pad_token_id = 128255
def prompt_generation(inputs):
    outputs = []
    for i in range(len(inputs)):
        temp = []
        temp.append({'role': 'system', 'content': "You are a personal judge of video game, your role is to judge which game is preferred by the user based on user's feedbacks of two games. Simply reply with the prefered game's full name, no need for explanation"})
        temp.append({
            'role': 'user',
            'content': f"I played two games {inputs[i][0]}, and {inputs[i][2]}. After playing, I gave reviews for both games as follow:\n{inputs[i][0]}: {inputs[i][1]}\n{inputs[i][2]}: {inputs[i][3]}"
        })
        outputs.append(temp)
    return outputs

In [143]:
game_list = ["Disco Elysium", "Lies of P", "Mafia III", "Total War: Warhammer III", "Resident Evil 3", "Phasmophobia", "Tom Clancy’s The Division 2", "The Witcher 3: Wild Hunt", "Fallout 76", "Cities: Skylines", "God of War: Ragnarok", "Just Cause 3", "Star Wars Jedi: Survivor", "Resident Evil Village", "Lightning Returns: Final Fantasy XIII", "Elden Ring", "Anno 2205", "Pay Day 2"]


In [90]:
# load a user's reviews
scores = []
texts = []
with open("data/synthesis_reviews/user_0.txt") as f:
    for line in f:
        line = line.strip()
        elements = line.split("):")
        scores.append(int(elements[0].split("(")[1].split("/")[0]))
        texts.append(elements[1].rstrip("\n"))
inputs = []
for i in range(len(game_list)-1):
    for j in range(i+1, len(game_list)):
        inputs.append((game_list[i], texts[i], game_list[j], texts[j]))
prompts = prompt_generation(inputs)

In [91]:
# Batch processing
prefered_games = []
for i in range(0, len(prompts), 32):
    msgs = prompts[i:i+32]
    inputs = tokenizer.apply_chat_template(
        msgs,
        tokenize = True,
        padding = True,
        truncation = False,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    )
    attention_mask = (inputs != tokenizer.pad_token_id).int()
    outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, attention_mask = attention_mask, use_cache = True)
    gen_idx = len(inputs[0])
    result = tokenizer.batch_decode(outputs[:, gen_idx:], skip_special_tokens = True)

    for k in range(len(result)):
        prefered_game = result[k]

        prefered_games.append(prefered_game)
    

In [93]:
from collections import Counter

counts = Counter(prefered_games)
ranking = counts.most_common()
print(dict(counts))

{'Disco Elysium': 17, 'Lies of P': 5, 'Total War: WARHAMMER III': 13, 'Phasmophobia': 9, 'Tom Clancy’s The Division 2': 9, 'The Witcher 3: Wild Hunt': 16, 'Cities: Skylines': 11, 'God of War: Ragnarok': 15, 'Just Cause 3': 8, 'Star Wars Jedi: Survivor': 5, 'Resident Evil Village': 10, 'Elden Ring': 14, 'Pay Day 2': 6, 'Resident Evil 3': 9, 'Mafia III': 2, 'Anno 2205': 3, 'Lightning Returns: Final Fantasy XIII': 1}


In [140]:
# load all reviews
counter = 0
all_scores = []
all_texts = []
scores = []
texts = []
with open("data/synthesis_reviews/reviews.txt") as f:
    for line in f:
        #print(counter, line)
        if counter == 0:
            counter += 1
            continue
        elif counter == 18:
            counter = 0
            line = line.strip()
            elements = line.split("):")
            scores.append(int(elements[0].split("(")[1].split("/")[0]))
            texts.append(elements[1].rstrip("\n"))
            all_scores.append(scores)
            all_texts.append(texts)
            scores = []
            texts = []
        else:
            line = line.strip()
            elements = line.split("):")
            scores.append(int(elements[0].split("(")[1].split("/")[0]))
            texts.append(elements[1].rstrip("\n"))
            counter += 1

all_scores

[[96, 70, 65, 85, 77, 76, 79, 87, 49, 84, 92, 73, 75, 84, 64, 91, 71, 74],
 [99, 76, 68, 83, 73, 72, 82, 89, 52, 82, 96, 72, 81, 80, 68, 93, 69, 81],
 [82, 87, 69, 85, 71, 70, 83, 96, 53, 84, 93, 74, 92, 78, 65, 100, 71, 81],
 [98, 77, 65, 87, 72, 71, 79, 90, 49, 86, 96, 74, 82, 79, 68, 94, 73, 74],
 [84, 77, 68, 88, 79, 78, 82, 90, 52, 87, 89, 72, 82, 86, 61, 94, 74, 81],
 [88, 84, 70, 90, 75, 74, 84, 96, 54, 89, 94, 79, 89, 82, 66, 100, 76, 79],
 [86, 79, 68, 78, 73, 72, 82, 89, 52, 77, 91, 68, 84, 80, 63, 93, 64, 84],
 [88, 89, 70, 82, 81, 80, 84, 98, 54, 81, 97, 76, 94, 88, 69, 100, 68, 80],
 [94, 80, 65, 83, 70, 69, 79, 90, 49, 82, 95, 69, 85, 77, 67, 94, 69, 78],
 [99, 80, 63, 89, 82, 81, 77, 90, 47, 88, 98, 71, 85, 89, 70, 94, 75, 71],
 [94, 73, 65, 93, 80, 79, 79, 89, 49, 92, 92, 75, 78, 87, 64, 93, 79, 72],
 [85, 77, 73, 93, 84, 83, 87, 97, 57, 92, 90, 87, 82, 91, 62, 100, 79, 76],
 [87, 70, 71, 86, 74, 73, 85, 87, 55, 85, 87, 74, 75, 81, 59, 91, 72, 84],
 [92, 84, 64, 88, 82,

In [144]:
def order_generation(counts):
    for g in game_list:
        if g not in counts.keys():
            counts[g] = 0
    ranking = [k for k, v in sorted(counts.items(), key=lambda x: x[1], reverse=True)]
    ranking_idx = [game_list.index(i) for i in ranking]
    return {
        "ranking" : ranking,
        "ranking_idx" : ranking_idx
    }

In [145]:
results = []
for x in range(50):
    texts = all_texts[x] 

    # Batch processing
    inputs = []
    for i in range(len(game_list)-1):
        for j in range(i+1, len(game_list)):
            inputs.append((game_list[i], texts[i], game_list[j], texts[j]))
    prompts = prompt_generation(inputs)
    
    prefered_games = []
    for i in range(0, len(prompts), 32):
        msgs = prompts[i:i+32]
        inputs = tokenizer.apply_chat_template(
            msgs,
            tokenize = True,
            padding = True,
            truncation = False,
            add_generation_prompt = True, # Must add for generation
            return_tensors = "pt",
        )
        attention_mask = (inputs != tokenizer.pad_token_id).int()
        outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, attention_mask = attention_mask, use_cache = True)
        gen_idx = len(inputs[0])
        result = tokenizer.batch_decode(outputs[:, gen_idx:], skip_special_tokens = True)
    
        for k in range(len(result)):
            prefered_game = result[k]
            prefered_games.append(prefered_game)

        counts = Counter(prefered_games)
        ranking = counts.most_common()
        result = order_generation(dict(counts))["ranking_idx"]

    results.append(result)

In [146]:
results

[[0, 7, 10, 3, 15, 9, 5, 13, 4, 2, 6, 11, 1, 12, 17, 16, 14, 8],
 [0, 7, 10, 15, 1, 3, 6, 9, 12, 13, 2, 5, 4, 17, 11, 16, 14, 8],
 [0, 7, 10, 1, 12, 15, 3, 9, 6, 13, 4, 2, 5, 17, 11, 16, 14, 8],
 [0, 7, 10, 3, 1, 12, 15, 9, 6, 4, 13, 11, 5, 17, 2, 16, 14, 8],
 [0, 7, 3, 10, 15, 1, 9, 12, 4, 5, 6, 13, 2, 17, 11, 16, 14, 8],
 [0, 7, 10, 15, 1, 12, 3, 9, 4, 6, 11, 13, 5, 2, 17, 16, 14, 8],
 [0, 7, 10, 1, 15, 12, 3, 6, 9, 13, 4, 5, 11, 17, 2, 16, 14, 8],
 [0, 7, 1, 10, 12, 15, 3, 4, 6, 13, 5, 9, 11, 17, 2, 16, 14, 8],
 [0, 7, 10, 15, 1, 12, 3, 9, 13, 11, 17, 6, 2, 4, 5, 16, 8, 14],
 [0, 7, 10, 15, 3, 13, 1, 4, 12, 9, 5, 11, 2, 17, 6, 16, 14, 8],
 [0, 7, 3, 9, 10, 13, 15, 4, 5, 11, 17, 6, 1, 12, 16, 2, 14, 8],
 [0, 7, 10, 15, 3, 1, 4, 9, 12, 13, 6, 2, 5, 11, 17, 16, 14, 8],
 [0, 7, 10, 15, 3, 6, 9, 13, 12, 17, 4, 2, 5, 11, 1, 14, 16, 8],
 [0, 7, 10, 12, 3, 1, 13, 15, 9, 4, 5, 6, 2, 11, 17, 16, 14, 8],
 [0, 7, 10, 15, 9, 1, 3, 12, 4, 13, 5, 6, 17, 2, 11, 14, 16, 8],
 [0, 10, 7, 12, 1, 15, 13

In [79]:
#----------------------------------

In [80]:
import numpy as np

def spearman_coefficient(list1, list2):
    n = len(list1)
    # 对两个列表分别进行排序，并获取排序后的索引
    rank1 = [sorted(list1).index(x) + 1 for x in list1]
    rank2 = [sorted(list2).index(x) + 1 for x in list2]
    # 计算排序后的差值平方和
    d_squared_sum = sum([(r1 - r2)**2 for r1, r2 in zip(rank1, rank2)])
    # 计算 Spearman 相关系数
    return 1 - (6 * d_squared_sum) / (n * (n**2 - 1))

In [81]:
def order_generation_gt(scores):
    counts = dict()
    for i in range(len(game_list)):
        counts[game_list[i]] = scores[i]
    ranking = [k for k, v in sorted(counts.items(), key=lambda x: x[1], reverse=True)]
    ranking_idx = [game_list.index(i) for i in ranking]
    return {
        "ranking" : ranking,
        "ranking_idx" : ranking_idx
    }

In [82]:
def order_generation(counts):
    for g in game_list:
        if g not in counts.keys():
            counts[g] = 0
    ranking = [k for k, v in sorted(counts.items(), key=lambda x: x[1], reverse=True)]
    ranking_idx = [game_list.index(i) for i in ranking]
    return {
        "ranking" : ranking,
        "ranking_idx" : ranking_idx
    }

In [94]:
result = order_generation(dict(counts))
tresult = order_generation_gt(scores)
print('True\t\t\t\t\tCalculated')
for i in range(len(result["ranking"])):
    print(f'{tresult["ranking"][i]}\t\t\t{result["ranking"][i]}')

True					Calculated
Disco Elysium			Disco Elysium
God of War: Ragnarok			The Witcher 3: Wild Hunt
Elden Ring			God of War: Ragnarok
The Witcher 3: Wild Hunt			Elden Ring
Total War: WARHAMMER III			Total War: WARHAMMER III
Cities: Skylines			Cities: Skylines
Resident Evil Village			Resident Evil Village
Tom Clancy’s The Division 2			Phasmophobia
Resident Evil 3			Tom Clancy’s The Division 2
Phasmophobia			Resident Evil 3
Star Wars Jedi: Survivor			Just Cause 3
Pay Day 2			Pay Day 2
Just Cause 3			Lies of P
Anno 2205			Star Wars Jedi: Survivor
Lies of P			Anno 2205
Mafia III			Mafia III
Lightning Returns: Final Fantasy XIII			Lightning Returns: Final Fantasy XIII
Fallout 76			Fallout 76


In [95]:
print(tresult["ranking_idx"])
print(result["ranking_idx"])

[0, 10, 15, 7, 3, 9, 13, 6, 4, 5, 12, 17, 11, 16, 1, 2, 14, 8]
[0, 7, 10, 15, 3, 9, 13, 5, 6, 4, 11, 17, 1, 12, 16, 2, 14, 8]


In [96]:
spearman_coefficient(result["ranking_idx"], tresult["ranking_idx"])

0.5397316821465428

In [120]:
# load a user's reviews
scores = []
texts = []
with open("data/synthesis_reviews/user_1.txt") as f:
    for line in f:
        line = line.strip()
        elements = line.split("):")
        scores.append(int(elements[0].split("(")[1].split("/")[0]))
        texts.append(elements[1].rstrip("\n"))
inputs = []
for i in range(len(game_list)-1):
    for j in range(i+1, len(game_list)):
        inputs.append((game_list[i], texts[i], game_list[j], texts[j]))
        
prompts = prompt_generation(inputs)

# Batch processing
prefered_games = []
for i in range(0, len(prompts), 64):
    msgs = prompts[i:i+64]
    inputs = tokenizer.apply_chat_template(
        msgs,
        tokenize = True,
        padding = True,
        truncation = False,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    )
    attention_mask = (inputs != tokenizer.pad_token_id).int()
    outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, attention_mask = attention_mask, use_cache = True)
    gen_idx = len(inputs[0])
    result = tokenizer.batch_decode(outputs[:, gen_idx:], skip_special_tokens = True)

    for k in range(len(result)):
        prefered_game = result[k]

        prefered_games.append(prefered_game)

from collections import Counter

counts = Counter(prefered_games)
ranking = counts.most_common()
print(dict(counts))

result = order_generation(dict(counts))
tresult = order_generation_gt(scores)
print('True\t\t\t\t\tCalculated')
for i in range(len(result["ranking"])):
    print(f'{tresult["ranking"][i]}\t\t\t{result["ranking"][i]}')

print("TRUE:",tresult["ranking_idx"])
print("PRED:",result["ranking_idx"])

spearman_coefficient(result["ranking_idx"], tresult["ranking_idx"])

{'Disco Elysium': 17, 'Lies of P': 11, 'The Witcher 3: Wild Hunt': 16, 'God of War: Ragnarok': 15, 'Star Wars Jedi: Survivor': 9, 'Resident Evil Village': 9, 'Elden Ring': 14, 'Total War: WARHAMMER III': 12, 'Mafia III': 4, 'Phasmophobia': 5, 'Tom Clancy’s The Division 2': 10, 'Cities: Skylines': 10, 'Just Cause 3': 4, 'Pay Day 2': 6, 'Resident Evil 3': 8, 'Lightning Returns: Final Fantasy XIII': 1, 'Anno 2205': 2}
True					Calculated
Disco Elysium			Disco Elysium
God of War: Ragnarok			The Witcher 3: Wild Hunt
Elden Ring			God of War: Ragnarok
The Witcher 3: Wild Hunt			Elden Ring
Total War: WARHAMMER III			Total War: WARHAMMER III
Tom Clancy’s The Division 2			Lies of P
Cities: Skylines			Tom Clancy’s The Division 2
Star Wars Jedi: Survivor			Cities: Skylines
Pay Day 2			Star Wars Jedi: Survivor
Resident Evil Village			Resident Evil Village
Lies of P			Resident Evil 3
Resident Evil 3			Pay Day 2
Phasmophobia			Phasmophobia
Just Cause 3			Mafia III
Anno 2205			Just Cause 3
Mafia III		

0.33333333333333337